In [ ]:
import requests
import json

In [ ]:
r = requests.get(url="https://www.cbioportal.org/api/health")
print (r.json())
print (r.status_code)

Part 1: Retrieve and display patient data. User must provide a valid patient and study Id. 
Example uses
patient: P-0000004 
study: msk_impact_2017

In [33]:
r = requests.get(url="https://www.cbioportal.org/api/studies/msk_impact_2017/patients/P-0000004/clinical-data")
x = r.json()
print (x)

[{'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'OS_STATUS', 'value': '0:LIVING'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SAMPLE_COUNT', 'value': '1'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SEX', 'value': 'Female'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SMOKING_HISTORY', 'value': 'Unknown'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'VITAL_STATUS', 'value': 'ALIVE'}]


Part 2: Find patients similar to a specified patient. Search within the same study, uses one criteria. 
Example uses
patient: P-0000004 
study: msk_impact_2017
attribute (not recorded across all studies): SMOKING_HISTORY

In [38]:
# get patient attribute
r = requests.get(url="https://www.cbioportal.org/api/studies/msk_impact_2017/patients/P-0000004/clinical-data?attributeId=SMOKING_HISTORY&projection=SUMMARY&pageSize=10000000&pageNumber=0&direction=ASC")
# parse for correct item (attribute)
# x in the following line returns a list with the keyval pairs, it needs to be taken out of the list to be parsed in the next line
x = r.json()
y = x[0]
pval = y['value']
print (y)
# Get list of patients from study
print ("patients")
rs = requests.get(url="https://www.cbioportal.org/api/studies/msk_impact_2017/clinical-data?attributeId=SMOKING_HISTORY&clinicalDataType=PATIENT&projection=SUMMARY&pageSize=10&pageNumber=0&direction=ASC")
clist = rs.json()
for a in clist:
    if a['value'] == y['value']:
        print (a['patientId'])

{'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SMOKING_HISTORY', 'value': 'Unknown'}
patients
P-0000004
P-0000030
